# Deploy churn model to an [Azure Function](https://azure.microsoft.com/en-us/services/functions/).

##Install required libs

In [0]:
!pip install azureml-contrib-functions
!pip install azureml-sdk

##Get the Azure ML Workspace

In [0]:
import azureml
from azureml.core import Workspace
import mlflow.azureml

workspace_name = '<YOUR-WORKSPACE-NAME>'
resource_group = '<YOUR-RESOURCE-GROUP>'
subscription_id = '<YOUR-SUBSCRIPTION-ID>'

workspace = Workspace.get(name = workspace_name,
                          resource_group = resource_group,
                          subscription_id = subscription_id)

##Get the registered model

In [0]:
from azureml.core.model import Model

model_name = 'churn-model'
model_azure = Model.list(workspace = workspace, name = model_name)[0]

##Add the requirements to an Environment

In [0]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
env = Environment(name='xgboost_env')

conda_dep = CondaDependencies('/dbfs/models/churn-prediction/conda.yaml')

# Define the packages needed by the model and scripts
conda_dep.add_pip_package("azureml-defaults")

# Adds dependencies to PythonSection of myenv
env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="/dbfs/models/churn-prediction/score.py",
                                   environment=env)

##Package the model using `azureml.contrib.function` to be used in the Azure Function

In [0]:
from azureml.contrib.functions import package
from azureml.contrib.functions import HTTP_TRIGGER

model_package = package(workspace, [model_azure], inference_config, functions_enabled=True, trigger=HTTP_TRIGGER)
model_package.wait_for_creation(show_output=True)

# Display the package location/ACR path
print(model_package.location)